<a href="https://colab.research.google.com/github/KORshinjoonghyeok/ML-Naver-sentiment-movie-corpus/blob/main/Naver_sentiment_movie_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload() # data 업로드

Saving ratings.txt to ratings.txt
Saving ratings_test.txt to ratings_test.txt
Saving ratings_train.txt to ratings_train.txt


In [ ]:
!pip install konlpy

In [11]:
import numpy as np
import pandas as pd

train_df = pd.read_csv('ratings_train.txt', sep = '\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [12]:
#0=부정
#1=긍정
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [13]:
import re

train_df = train_df.fillna(' ')
#숫자를 공백으로 변경하는 정규식
train_df['document'] = train_df['document'].apply( lambda x : re.sub(r'\d+', ' ', x))
#테스트 데이터 로딩및 정규화
test_df = pd.read_csv('ratings_test.txt', sep = '\t')
test_df = test_df.fillna(' ')
test_df['documnet'] = test_df['document'].apply( lambda x :re.sub(r'\d+', ' ', x))
#불필요한 id칼럼 drop
train_df.drop('id', axis=1, inplace=True)
test_df.drop('id', axis=1, inplace=True)

In [14]:
from konlpy.tag import Twitter

twitter = Twitter()

def tw_tokenizer(text):
  #텍스트를 형태소 단어로 토크나이저후 리스트 형태로 리턴
  tokens_ko = twitter.morphs(text)
  return tokens_ko

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

#Twitter객체의 morphs()객체를 이용한 tokenizer. ngram_range(1,2)
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [30]:
#로지스틱 회귀 이용 감성분석 분류
lg_clf = LogisticRegression(random_state=0)

#파라미터 최적화 GridSearchCV
params = {'C': [1,3.5,4,5.5,10]}

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
grid_cv = GridSearchCV(lg_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train, train_df['label'])
print(grid_cv.best_params_, round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  1.2min finished


{'C': 3.5} 0.8593


In [33]:
from sklearn.metrics import accuracy_score

#학습 데이터를 적용한 TfidVectorizer를 이용 테스트 데이터를 TF-IDF값으로 피쳐변환
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

#best params이용
best_estimator = grid_cv.best_estimator_
pred = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ', accuracy_score(test_df['label'],pred))

Logistic Regression 정확도:  0.86102
